## Illustrative example implementing COSMOS ideas on Korda2020 optimisation problem

In the paper by Korda2020 we are solving 
$$ \min_{\lambda_i, g_i} \left\|
  \begin{bmatrix}
      h_i(x^1(0)) & h_i(x^2(0)) & \cdots \\ 
      h_i(x^1(1)) & h_i(x^2(1)) & \cdots \\ 
      h_i(x^1(2)) & h_i(x^2(2)) & \cdots \\ 
      \vdots & \vdots & \ddots %\\ h_i(x^1(N)) \\ h_i(x^2(0)) \\ \vdots
  \end{bmatrix}- 
  \begin{bmatrix}
      1 & 1 & 1 & \cdots \\
      \lambda_{1} & \lambda_{2} & \lambda_{3} & \cdots \\
      \lambda_{1}^2 & \lambda_{2}^2 & \lambda_{3}^2 & \cdots \\
      \vdots & \vdots & \vdots & \ddots
  \end{bmatrix}
  \begin{bmatrix}% End of phantom section for vertical brace alignment
      g_{1} (x^1(0)) & g_{1} (x^2(0)) & \cdots \\ 
      g_{2} (x^1(0)) & g_{2} (x^2(0)) & \cdots \\
      \vdots & \vdots & \ddots
  \end{bmatrix} \right\| $$

That is, given a matrix $Y$ we seek a decomposition into a product of a Vandermonde matrix $\Lambda$ and a matrix of initial states $G$, formulated as a least-squares problem $\| Y - \Lambda G \|$. The idea is to turn this problem from a nonconvex problem into a convex problem with rank constraint, and apply the ideas in COSMOS to this work. 
Defining $D_\lambda = \text{diag}(\lambda_1, \lambda_2, \dots)$, and $\overline \Lambda$ as the matrix $\Lambda$, but shifted one place up, we obtain that $\overline \Lambda = \Lambda D_{\lambda}$. These relationships can be written in terms of a rank constraint, for which we introduce
$$ H = \begin{bmatrix} \hat M & \Lambda & \overline \Lambda \\ \hat G & I_{N_g} & D_\lambda \end{bmatrix}. $$

By the Schur decomposition, the rank of the matrix $\hat M - \Lambda I_{N_g} \hat G$ is equivalent to rank of the left $2 \times 2$ block submatrix of $H$. Similarly, the introduced relationship between $\Lambda$ and its shifted matrix $\overline \Lambda$, gives the latter columns the same rank. Hence we obtain the optimisation problem
$$ \min \| Y - \hat M \| $$
subject to $\text{rank}(H) = N_g$

In [96]:
import cvxpy as cvx
import numpy as np
np.random.seed(123)

def H_Theta(M, Theta, ThetaShift, X, Id, L):
    '''Function for constructing the blockmatrix in the rank constraint optimisation problem
    Using the input matrices, the following block matrix is constructed:
    | M  Theta  ThetaShift |
    | X    Id     L        |
    '''
    return cvx.bmat([[M, Theta, ThetaShift], [X, Id, L]])


N = 20                             # Number of rows of measurement data Y, i.e. measurement sequence length
N_t = 2                           # Number of columns of measurement data Y, i.e. number of measurement sequences
N_g = 3                            # Rank used as rank constraint, and to construct measurement data Y
Id = np.eye(N_g)                   # Usefull identity matrix
# Total number of optimisation variables = N_t * N_g + N_g

# Define Y as decomposed product of a VanderMonde matrix and random initial states.
Y = np.vander([0.9, 0.95, 0.8], N, increasing=True).T @ np.random.rand(N_g, N_t) 

# Initialise U and V as empty matrices
U_j = [np.zeros((N + N_g, N_g))]
V_j = [np.zeros((N_t + 2 * N_g, N_g))]

# Initialise CVX optimisation problem
Mhat = cvx.Variable(Y.shape)
Ghat = cvx.Variable((N_g, N_t))
ThetaHat = cvx.Variable((N,N_g))
ThetaHatShift = cvx.Variable((N, N_g))
Lhat = cvx.diag(cvx.Variable(N_g, complex=False))      

# Constraints: 
#        Requirement that first row of Theta[0,:] = [ 1, 1, 1, ...]
#        Requirement that second row of Theta[1,:] = ThetaShift[0,:]
#        Requirement that second row of Theta[1,:] = [lambda_1, lambda_2, ...]
cons = [ThetaHat[0,:] == np.ones((N_g,)), ThetaHat[1:,:] == ThetaHatShift[:-1,:], ThetaHat[1,:] == cvx.diag(Lhat)]

# Construct rank constraint matrix
Fullmatrix = H_Theta(Mhat, ThetaHat, ThetaHatShift, Ghat, Id, Lhat)

# Initialise variables that change on optimisation iteration
U = cvx.Parameter((N + N_g, N_g), complex=False)
V = cvx.Parameter((N_t + 2 * N_g, N_g), complex=False)
rho = cvx.Parameter(pos=True)

# Define objective function and problem handle
obj = cvx.norm(Y - Mhat,2) +  rho * ( cvx.norm(Fullmatrix, "nuc") - (cvx.trace(U.T @ Fullmatrix @ V))) 
prob = cvx.Problem(cvx.Minimize(obj), cons)

# Initialise optimisation iteration
i = 0
eps = 1
rho_new = 0.01       # Initial value for rho
mu = 1.02            # Growth factor for rho
rho_max = 10     # Maximum value for rho
prev_value = 1e5

# Iterate while error is large and number of iterations is small enough
while eps > 1e-4 and i < 10:
    rho_new = min(mu * rho_new, rho_max)
    
    # Set CVX problem parameters
    rho.value = rho_new
    U.value = U_j[i]
    V.value = V_j[i]
    
    # Solve optimisation problem iteration using cvx
    prob.solve(verbose=False)

    # Compute SVD of rank-constraint matrix value
    Res = Fullmatrix.value # H_Theta_num(M, Theta, ThetaShift, X, Id, L)
    U1, s, V1 = np.linalg.svd(Res, full_matrices=True)

    U_j.append(U1[:,:N_g])
    V_j.append(V1[:N_g,:].T)
 
    # Update error value, and print update
    eps = abs((obj.value - prev_value) / obj.value)
    prev_value = obj.value
    i = i + 1
    print(f'Iteration {i} completed with error {obj.value:.3f} and rho={rho_new:.2f}')
    
# Extract solutions from optimisation procedure
M = Mhat.value
G = Ghat.value
Theta = ThetaHat.value
ThetaShift = ThetaHatShift.value
L = Lhat.value 

print()
print(f'------------------ Finished optimisation procedure with status word: {prob.status} ----------------------------')
print(f'Found eigenvalue sequence                                  {np.array2string(np.diag(L), precision=2)}')
# print(Theta)
# print(ThetaShift)
print(f"Final nuclear norm of rank-constraint matrix               {np.linalg.norm(Res, 'nuc'):.3f}")
print(f'Least-square norm error ||Y - M||                          {np.linalg.norm(Y - M, 2):.3f}')
print(f'Least-square norm error ||Y - L*G||                        {np.linalg.norm(Y - np.vander(np.diag(L), N, increasing=True).T @ G, 2):.3f}') 
# print(Res)

Iteration 1 completed with error 0.087 and rho=0.01
Iteration 2 completed with error 0.013 and rho=0.01
Iteration 3 completed with error 0.012 and rho=0.01
Iteration 4 completed with error 0.005 and rho=0.01
Iteration 5 completed with error 0.003 and rho=0.01
Iteration 6 completed with error 0.003 and rho=0.01
Iteration 7 completed with error 0.003 and rho=0.01
Iteration 8 completed with error 0.003 and rho=0.01
Iteration 9 completed with error 0.002 and rho=0.01
Iteration 10 completed with error 0.001 and rho=0.01

------------------ Finished optimisation procedure with status word: optimal ----------------------------
Found eigenvalue sequence                                  [0.16 0.44 0.9 ]
Final nuclear norm of rank-constraint matrix               9.002
Least-square norm error ||Y - M||                          0.000
Least-square norm error ||Y - L*G||                        0.180


In [97]:
print(np.vander(np.diag(L), N, increasing=True).T @ G)
print(Y - np.vander(np.diag(L), N, increasing=True).T @ G)
# print(Y - M)

[[1.64285396 1.26060982]
 [1.4296162  1.10827841]
 [1.20396493 1.02090597]
 [1.03867957 0.93580503]
 [0.91473971 0.85180866]
 [0.81532731 0.77212698]
 [0.73130846 0.69838918]
 [0.6580309  0.63101345]
 [0.59302736 0.56983526]
 [0.5348592  0.51445446]
 [0.48258007 0.46439668]
 [0.43549216 0.41918343]
 [0.39303482 0.3783605 ]
 [0.35473268 0.34150805]
 [0.3201702  0.30824278]
 [0.28897834 0.27821677]
 [0.26082666 0.25111515]
 [0.23541806 0.22665336]
 [0.21248493 0.20457437]
 [0.19178594 0.18464613]]
[[-6.43521764e-05 -4.92582913e-05]
 [-1.17098802e-02  1.14811901e-02]
 [ 2.53686845e-02 -2.07833993e-02]
 [ 3.19113398e-02 -3.78954431e-02]
 [ 2.16801432e-02 -4.17189116e-02]
 [ 7.21969736e-03 -3.79241841e-02]
 [-5.81504420e-03 -3.01411691e-02]
 [-1.56094408e-02 -2.04183699e-02]
 [-2.20154601e-02 -9.92269440e-03]
 [-2.54942778e-02  6.55976927e-04]
 [-2.66595483e-02  1.08971618e-02]
 [-2.60977090e-02  2.05425837e-02]
 [-2.43083506e-02  2.94378507e-02]
 [-2.16939303e-02  3.74983232e-02]
 [-1.8568